In [1]:
## NowPlaying Cultural Sentiment Data
"""
cs_header = ["hashtag", "AFINN_min", "AFINN_max", "AFINN_sum", "AFINN_avg", "OpinionLex_min", "OpinionLex_max", "OpinionLex_sum", "OpinionLex_avg", "Sentistrength_min", "Sentistrength_max", "Sentistrength_sum", "Sentistrength_avg", "Vader_min", "Vader_max", "Vader_sum", "Vader_avg", "SentimentHashtag_min", "SentimentHashtag_max", "SentimentHashtag_sum", "SentimentHashtag_avg"]
np_cs = pd.read_csv('np_cultural_sentiment.csv',names = cs_header)
"""
#IMPORTING Cultural Sentiment Data to MongoDB
"""
np_cs_json = json.loads(np_cs.to_json(orient="records"))
db.cultural_sentiment.drop()
db.cultural_sentiment.insert_many(np_cs_json)
"""
## Transformation of NowPlaying Cultural Tweets to DataframeStructure
"""
output = []
with open("np_cultural.json") as f:
    for line in f:     
        output.append(json.load|s(line))
np_c = pd.DataFrame(output)
"""
#IMPORTING Cultural Tweets to MongoDB
"""
db.cultural_tweet.drop()
records = json.loads(np_c.T.to_json()).values()
db.cultural_tweet.insert_many(records)
"""

## Transformation and Importing of Playlist Data from NowPlaying to MongoDB
"""
np_pl = pd.read_csv("playlist.csv", names=["playlist_id","track_id","playlist_name"])
np_pl_json = json.loads(np_pl.to_json(orient="records"))
db.playlist.drop()
db.playlist.insert_many(np_pl_json)
"""

'\nnp_pl = pd.read_csv("playlist.csv", names=["playlist_id","track_id","playlist_name"])\nnp_pl_json = json.loads(np_pl.to_json(orient="records"))\ndb.playlist.drop()\ndb.playlist.insert_many(np_pl_json)\n'

# CULTURAL #NOWPLAYING DATA
*********************************

This dataset contains data describing the listening events of users (extracted from the #nowplaying dataset), the emotion extracted from the hashtags used in the according tweets and information about the location of the user.


TWITTER AND TRACK DATA
-------------------------------
The data regarding the listening events is contained in np_cultural.zip and is encoded as json. It holds the following information:

- id: the id of the underlying tweet [*]
- user_id: the id of the user who sent the tweet (MD5 of it)
- user_lang: The BCP 47 code for the user’s self-declared user interface language. [*]
- user_time_zone: [*]
- text: actual content of the tweet [*]
- tweet_lang: language of the tweet (as detected by Twitter; BCP 47 language identifier corresponding to the machine-detected language of the Tweet text, or und if no language could be detected.) [*]
- geo: Deprecated version of coordinates (however, we deal with data stemming from before this API change, therefore we still include it; cf. coordinates for a description)
- coordinates: Represents the geographic location of this Tweet as reported by the user or client application. The inner coordinates array is formatted as geoJSON (longitude first, then latitude). [*]
- place: When present, indicates that the tweet is associated (but not necessarily originating from) a Place. [*]
- created_at: time the tweet was sent. [*]
- source: Utility used to post the Tweet, as an HTML-formatted string. [*]
- track_title: title of the track the user tweeted about
- track_id: the unique id of the track (from #nowplaying dataset) 
- artist_name: name of artist performing the track
- artist_id: the unique id of the artist (from #nowplaying dataset) 
- hashtags: list of hashtags used in the tweet.

[*] for further information about the information gathered from Twitter, please consult https://dev.twitter.com/overview/api/tweets

Please note that we do only add key-value pairs for geo/coordinates/place information if this information was provided by the Twitter API (i.e., missing keys signal that this information is not available for the given tweet).


SENTIMENT DATA
-------------------------------
The data regarding the hashtag's sentiment (if any could be obtained) is contained in np_cultural_sentiment.csv and is formatted as csv. The sentiment score was obtained by applying a set of well-known sentiment dictionaries. The sentiment scores are scaled between 0 and 1 (very negative to very positive). For each dictionary, we list the minimum, maximum, sum and average sentiment score across all hashtags used within every tweet (most tweets only feature a single hashtag we can assign a sentiment value to)
It contains the following information (in this particular order):
- name of the hashtag
- AFINN dictionary (min, max, sum, avg)
- Opinion Lexicon (min, max, sum, avg)
- Sentistrength Lexicon (min, max, sum, avg)
- Vader (min, max, sum, avg)
- Sentiment Hashtag Lexicon (min, max, sum, avg)


Please note that we only added hashtags for which we could obtain a sentiment value from at least one sentiment dictionary.

In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np 
import json
import csv
import pprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from difflib import SequenceMatcher


In [2]:
#Call Mongo Instance
client = MongoClient()
db = client.now_playing

#Spotify Instance
client_credentials_manager = SpotifyClientCredentials(client_id="7736d10450e04c5f9e302bb07a4f6cf7", client_secret="a11e29bc5c324ddeb19fc6249d303814",)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### Importing the MusicBrainz Mapping of artists and tracks and tracks from cultural dataset

`mongoimport -d now_playing -c mb_tracks --type csv --file mb_tracks.csv --fields "np_id,mb_track_id"`

`mongoimport -d now_playing -c mb_artists --type csv --file mb_artists.csv --fields "np_id,mb_artist_id"`

`mongoimport -d now_playing -c np_cultural_mb_tracks --type csv --file np_cultural_mb_tracks.csv --headerline
`

In [3]:
user_ids = db.cultural_tweet.distinct("user_id")
type(user_ids)


list

In [34]:
#db.cultural_tweet.find({"hashtag": "nobeats"})
user_tweets = db.cultural_tweet.find({"user_id": '1c10f9788fdcc4baf6cf6a2631fe78bc12102418'})
#TOTAL Number of Tweets
print("Total number of Tweets: "+ str(db.cultural_tweet.find({}).count()))
print("Total number of Songs: " + str(len(db.cultural_tweet.distinct("track_title"))))
#Get Distinct User Ids
print("Total number of Users: " + str(len(user_ids)))
print("Total number of Tweets with duration: "+ str(db.cultural_tweet.find({"duration_ms" : {"$exists": True}}).count()))

print("Total number of Tweets without duration: "+ str(db.cultural_tweet.find({"duration_ms" : {"$exists": False}}).count()))

Total number of Tweets: 564301
Total number of Songs: 39776
Total number of Users: 9431
Total number of Tweets with duration: 276883
Total number of Tweets without duration: 287418


In [5]:
first = user_ids[0]
user_songs = db.cultural_tweet.find({"user_id": first})
initial_song = user_songs[0]
initial_date = initial_song["created_at"]

In [6]:
#db.mb_tracks.find({"np_id": initial_song["track_id"]}).count()
#for song in user_songs:
#    print(str(song["track_title"]))
#    print(str(song["track_id"]))
    
print(len(db.cultural_tweet.distinct("track_id")))
print(len(db.cultural_tweet.distinct("track_title")))

51045
39776


In [7]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
def convertMillis(millis):
     seconds = (millis/1000)%60
     minutes = (millis/(1000*60))%60
     return str(int(minutes)) + ":" +str(int(seconds))


In [8]:
"""
search_query = initial_song["track_title"].lower() +" "+ initial_song["artist_name"].lower()
results = sp.search(q=search_query,limit=5)
print("Looking for: "+search_query)
print("Querying...")
for track in results["tracks"]["items"]:
    artist_names = list(map(lambda x: x["name"],track["artists"]))
    print(" ".join(artist_names))
    print("Found: " + str(track["name"]) + " "+str(track["artists"][0]["name"]))
    print(convertMillis(track["duration_ms"]))
    result_matcher = track["name"].lower() + " " + track["artists"][0]["name"].lower()
    print("Similarity Score: " + str((similar(search_query, result_matcher))))
""

'\nsearch_query = initial_song["track_title"].lower() +" "+ initial_song["artist_name"].lower()\nresults = sp.search(q=search_query,limit=5)\nprint("Looking for: "+search_query)\nprint("Querying...")\nfor track in results["tracks"]["items"]:\n    artist_names = list(map(lambda x: x["name"],track["artists"]))\n    print(" ".join(artist_names))\n    print("Found: " + str(track["name"]) + " "+str(track["artists"][0]["name"]))\n    print(convertMillis(track["duration_ms"]))\n    result_matcher = track["name"].lower() + " " + track["artists"][0]["name"].lower()\n    print("Similarity Score: " + str((similar(search_query, result_matcher))))\n'

In [45]:
final_result = dict()
score_result = dict()
name_found_result = dict()
searched_name = dict()
#Get Spotify's song duration
track_list = db.cultural_tweet.find({"duration_ms" : {"$exists" : False }}).distinct("track_id")
track_list_len = len(track_list)
searched = 0
for track in track_list:
    song_name = db.cultural_tweet.find_one({"track_id": track }, {"track_title" : 1 , "artist_name": 1})
    search_query = song_name["track_title"].lower() + " " + song_name["artist_name"].lower()
    search_result = sp.search(q = search_query, limit= 3, type="track")
    searched += 1
    score_result[track] = 0
    for t in search_result["tracks"]["items"]:
        artists = " ".join(list(map(lambda x: x["name"],t["artists"])))
        result_matcher = t["name"].lower() + " " + artists.lower()
        similarity_score = similar(search_query,result_matcher)
        if(similarity_score > score_result[track]):
            print("Found " + result_matcher + "  => Similarity Score: " + str(similarity_score))
            final_result[track] = t["duration_ms"]
            score_result[track] = similarity_score
            name_found_result[track] = result_matcher
            searched_name[track] = search_query
            break
    print("Completed " + str(searched) +" out of " + str(track_list_len))         


Completed 1 out of 38385
Found what a feeling (flashdance) - clubhouse album mix global deejays  => Similarity Score: 0.623655913978
Completed 2 out of 38385
Found bat country avenged sevenfold  => Similarity Score: 0.277777777778
Completed 3 out of 38385
Completed 4 out of 38385
Found undone - the sweater song weezer  => Similarity Score: 0.577777777778
Completed 5 out of 38385
Found under your spell the birthday massacre  => Similarity Score: 0.691358024691
Completed 6 out of 38385
Found ooh la la (from "the smurfs 2") britney spears  => Similarity Score: 0.685714285714
Completed 7 out of 38385
Found gold digger kanye west jamie foxx  => Similarity Score: 0.8
Completed 8 out of 38385
Found k.y.h.u. - noisecontrollers rmx wildstylez  => Similarity Score: 0.526315789474
Completed 9 out of 38385
Found (you want to) make a memory - pop album version bon jovi  => Similarity Score: 0.564102564103
Completed 10 out of 38385
Found oh boy cam'ron juelz santana  => Similarity Score: 0.421052631

Completed 106 out of 38385
Found running back (feat. lil wayne) wale lil wayne  => Similarity Score: 0.25
Completed 107 out of 38385
Found brazil ray hamilton orchestra  => Similarity Score: 0.521739130435
Completed 108 out of 38385
Found if i ruled the world (imagine that) nas ms. lauryn hill  => Similarity Score: 0.607594936709
Completed 109 out of 38385
Found the sound of san francisco - progressive album mix global deejays  => Similarity Score: 0.77358490566
Completed 110 out of 38385
Completed 111 out of 38385
Completed 112 out of 38385
Completed 113 out of 38385
Found neverland zendaya  => Similarity Score: 0.722222222222
Completed 114 out of 38385
Found say yeah - radio edit wiz khalifa  => Similarity Score: 0.68
Completed 115 out of 38385
Found ? mf doom kurious  => Similarity Score: 0.5625
Completed 116 out of 38385
Found ashes to ashes david bowie  => Similarity Score: 0.653846153846
Completed 117 out of 38385
Found just dance lady gaga colby o'donis  => Similarity Score: 0.7

Found let me think about it (in the style of ida corr vs fedde le grand) [karaoke version] ameritz digital karaoke  => Similarity Score: 0.615384615385
Completed 204 out of 38385
Found holding back the years - 2008 remastered version simply red  => Similarity Score: 0.439024390244
Completed 205 out of 38385
Found the tide is high (get the feeling) [radio mix] atomic kitten  => Similarity Score: 0.666666666667
Completed 206 out of 38385
Completed 207 out of 38385
Found redline leadfoot karl agell of coc phil swisher of coc  => Similarity Score: 0.457142857143
Completed 208 out of 38385
Completed 209 out of 38385
Found no love lost (extended mix) cece rogers syke 'n' sugarstarr  => Similarity Score: 0.674157303371
Completed 210 out of 38385
Found doomsday mf doom mf grimm  => Similarity Score: 0.486486486486
Completed 211 out of 38385
Found put on jeezy kanye west  => Similarity Score: 0.521739130435
Completed 212 out of 38385
Found smooth florida georgia line  => Similarity Score: 0.45


Found candy store (feat. jessica keenan wynn, alice lee & elle mclemore) various artists jessica keenan wynn alice lee elle mclemore  => Similarity Score: 0.254777070064
Completed 305 out of 38385
Found funeralopolis electric wizard  => Similarity Score: 0.633333333333
Completed 306 out of 38385
Completed 307 out of 38385
Found saturday night - radio mix whigfield  => Similarity Score: 0.8
Completed 308 out of 38385
Found samurai (go hard ) - original mix r3hab  => Similarity Score: 0.297872340426
Completed 309 out of 38385
Completed 310 out of 38385
Found 2 hot 4 u (feat. $uicdeboy$) fat nick $uicideboy$  => Similarity Score: 0.454545454545
Completed 311 out of 38385
Found i am the fly - 2006 remastered version wire  => Similarity Score: 0.566666666667
Completed 312 out of 38385
Found what you want - radio edit dmx sisqo  => Similarity Score: 0.641509433962
Completed 313 out of 38385
Found sussudio - 2016 remastered phil collins  => Similarity Score: 0.7
Completed 314 out of 38385
Fou

Found teardrops on my guitar - radio single remix taylor swift  => Similarity Score: 0.769230769231
Completed 403 out of 38385
Found where'd you go (feat. holly brook & jonah matranga) fort minor holly brook jonah matranga  => Similarity Score: 0.424778761062
Completed 404 out of 38385
Found fade to grey visage  => Similarity Score: 0.625
Completed 405 out of 38385
Completed 406 out of 38385
Completed 407 out of 38385
Found let it go - single version demi lovato  => Similarity Score: 0.71186440678
Completed 408 out of 38385
Found rick james - boomrock saints superfreak remix kid wobble boomrock saints  => Similarity Score: 0.387096774194
Completed 409 out of 38385
retrying ...1secs
Found the seed (2.0) the roots cody chesnutt  => Similarity Score: 0.642857142857
Completed 410 out of 38385
Found this perfect day - single version the saints  => Similarity Score: 0.760563380282
Completed 411 out of 38385
Completed 412 out of 38385
Found mob justice - original mix foreign concept  => Simil

...

Completed 6549 out of 38385
Found hey nas nas kelis claudette ortiz  => Similarity Score: 0.5
Completed 6550 out of 38385
Found would it matter - bonus track skillet  => Similarity Score: 0.766666666667
Completed 6551 out of 38385
Found bad boyz (feat. barrington levy) shyne barrington levy  => Similarity Score: 0.215384615385
Completed 6552 out of 38385
Found morning train (nine to five) sheena easton  => Similarity Score: 0.58064516129
Completed 6553 out of 38385
Found open letter, my one temptation interlude the temptations  => Similarity Score: 0.634146341463
Completed 6554 out of 38385
Found hungry for heaven - remastered dio  => Similarity Score: 0.763636363636
Completed 6555 out of 38385
Found sexy bitch (feat. akon) david guetta akon  => Similarity Score: 0.71875
Completed 6556 out of 38385
Completed 6557 out of 38385
Completed 6558 out of 38385
Found girlfriend avril lavigne  => Similarity Score: 0.561403508772
Completed 6559 out of 38385
Found ain't no sunshine (karaoke versi

Completed 6642 out of 38385
Found all that i'm living for vitamin string quartet  => Similarity Score: 0.6
Completed 6643 out of 38385
Found who's that chick ? (feat. rihanna) david guetta rihanna  => Similarity Score: 0.589743589744
Completed 6644 out of 38385
Found roses never cry - radio edit heatbeat  => Similarity Score: 0.786885245902
Completed 6645 out of 38385
Found jump up major lazer leftside supa hype  => Similarity Score: 0.666666666667
Completed 6646 out of 38385
Found the musical box genesis  => Similarity Score: 0.6
Completed 6647 out of 38385
Found all cats are grey - remastered version the cure  => Similarity Score: 0.712328767123
Completed 6648 out of 38385
Completed 6649 out of 38385
Completed 6650 out of 38385
Found victim - rerecorded candi staton  => Similarity Score: 0.745098039216
Completed 6651 out of 38385
Found she was so flyy feat. kardinal offishall dj jazzy jeff  => Similarity Score: 0.698795180723
Completed 6652 out of 38385
Completed 6653 out of 38385
Fo

Found full time freaks white fang  => Similarity Score: 0.578947368421
Completed 6738 out of 38385
Found blood like cream red fang  => Similarity Score: 0.576923076923
Completed 6739 out of 38385
Found eleven echoes of autumn, 1965 (echoes i): xi. eco 11. adagio; ("like a prayer") george crumb jamie van eyck jacob greenberg international contemporary ensemble  => Similarity Score: 0.118343195266
Completed 6740 out of 38385
Completed 6741 out of 38385
Found a bit of finger / sleeping village / warning - remastered version black sabbath  => Similarity Score: 0.550458715596
Completed 6742 out of 38385
Found party like a rockstar trina  => Similarity Score: 0.682926829268
Completed 6743 out of 38385
Found mmmbop - single version hanson  => Similarity Score: 0.604651162791
Completed 6744 out of 38385
Found module genesis minus 0 plus 1  => Similarity Score: 0.454545454545
Completed 6745 out of 38385
Found ace of wands - 2005 - remaster steve hackett  => Similarity Score: 0.742857142857
Comp

Completed 6826 out of 38385
Found two of us on the run lucius  => Similarity Score: 0.454545454545
Completed 6827 out of 38385
Found malfeitor watain  => Similarity Score: 0.620689655172
Completed 6828 out of 38385
Found that summer brooks jefferson  => Similarity Score: 0.692307692308
Completed 6829 out of 38385
Found la negra tomasa - bilongo - versión tropical caifanes  => Similarity Score: 0.623376623377
Completed 6830 out of 38385
Found rotation future  => Similarity Score: 0.625
Completed 6831 out of 38385
Completed 6832 out of 38385
Found went to poland - club mix edit bacefook  => Similarity Score: 0.741935483871
Completed 6833 out of 38385
Found paradise coldplay  => Similarity Score: 0.486486486486
Completed 6834 out of 38385
Completed 6835 out of 38385
Found away from here - us version the enemy uk  => Similarity Score: 0.75
Completed 6836 out of 38385
Found sovereignty red tank!  => Similarity Score: 0.484848484848
Completed 6837 out of 38385
Found liberian girl - 2012 rema

TypeError: 'NoneType' object has no attribute '__getitem__'

In [ ]:
for track in final_result:
    if score_result[track] > 0.80:
        db.cultural_tweet.update_many({"track_id": track},
                                {"$set" : {
                                    "duration_ms" : final_result[track]
                                }})
    


In [ ]:
clean_dic = {track: final_result[track] for track in final_result if score_result[track] <= 0.80}

In [ ]:
print("Songs with duration " + str(len(db.cultural_tweet.find({"duration_ms" : {"$exists" : True }}).distinct("track_id"))))
print("Songs without duration " + str(len(db.cultural_tweet.find({"duration_ms" : {"$exists" : False}}).distinct("track_id"))))
print("Songs in total " + str(len(db.cultural_tweet.distinct("track_id"))))


In [46]:
search_result = sp.search(q = "Despacito", limit= 3, type="track")

In [55]:
for track in search_result["tracks"]["items"]:
    print(track["id"])

6rPO02ozF3bM7NnOV4h6s2
7CUYHcu0RnbOnMz4RuN07w
4vxA3aI7l73i0Hi819OQhH


In [54]:
test[0]

{u'album': {u'album_type': u'single',
  u'artists': [{u'external_urls': {u'spotify': u'https://open.spotify.com/artist/4V8Sr092TqfHkfAA5fXXqG'},
    u'href': u'https://api.spotify.com/v1/artists/4V8Sr092TqfHkfAA5fXXqG',
    u'id': u'4V8Sr092TqfHkfAA5fXXqG',
    u'name': u'Luis Fonsi',
    u'type': u'artist',
    u'uri': u'spotify:artist:4V8Sr092TqfHkfAA5fXXqG'},
   {u'external_urls': {u'spotify': u'https://open.spotify.com/artist/4VMYDCV2IEDYJArk749S6m'},
    u'href': u'https://api.spotify.com/v1/artists/4VMYDCV2IEDYJArk749S6m',
    u'id': u'4VMYDCV2IEDYJArk749S6m',
    u'name': u'Daddy Yankee',
    u'type': u'artist',
    u'uri': u'spotify:artist:4VMYDCV2IEDYJArk749S6m'}],
  u'available_markets': [u'CA', u'MX', u'US'],
  u'external_urls': {u'spotify': u'https://open.spotify.com/album/3Gq2Dme9nesdgoqNNlcN8O'},
  u'href': u'https://api.spotify.com/v1/albums/3Gq2Dme9nesdgoqNNlcN8O',
  u'id': u'3Gq2Dme9nesdgoqNNlcN8O',
  u'images': [{u'height': 640,
    u'url': u'https://i.scdn.co/image/e